In [3]:
import av
import cv2
import numpy as np
import pandas as pd

In [15]:
images_av = []
images_cv2 = []


In [16]:
%%timeit
container = av.open("./videos/WUzgd7C1pWA.mp4")
for frame in container.decode(video=0):
    images_av.append(frame.to_rgb().to_ndarray())



455 ms ± 29.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
cap = cv2.VideoCapture("./videos/WUzgd7C1pWA.mp4")
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret is True:
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        images_cv2.append(frame)
    else:
        break
cap.release()

188 ms ± 24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
images_av = []
images_cv2 = []
container = av.open("./videos/WUzgd7C1pWA.mp4")
for frame in container.decode(video=0):
    images_av.append(frame.to_rgb().to_ndarray())

cap = cv2.VideoCapture("./videos/WUzgd7C1pWA.mp4")
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret is True:
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        images_cv2.append(frame)
    else:
        break
cap.release()

In [19]:

print(len(images_av), len(images_cv2))

327 327


In [1]:
import torchvision
torchvision.set_video_backend("video_reader")

In [2]:
vframes, _, _ = torchvision.io.read_video("./videos/WUzgd7C1pWA.mp4")
vframes.size()

/Users/bjuncek/bin/conda/envs/video_benchmark/lib/python3.7/site-packages/torchvision-0.8.0a0+9b80465-py3.7-macosx-10.9-x86_64.egg/torchvision/io/_video_opt.py:479: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "


torch.Size([327, 256, 340, 3])

# Simple measurement of python overhead

In [1]:
# first, import torch
import torch

In [2]:
%%timeit
# torch multitensor
a = torch.zeros(1000, 3, 112, 112)

70.4 ms ± 5.56 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
%%timeit
# for loop with array allocation
test = []
for i in range(1000):
    test.append(torch.zeros(3, 112, 112))

93 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%timeit
# for loop with array allocation and stacking
test = []
for i in range(1000):
    test.append(torch.zeros(3, 112, 112))
a = torch.stack(test)


179 ms ± 4.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Exploring DECORD

With GPU and CPU support (or at least I hope so)

In [33]:
import torchvision
torchvision.set_video_backend("video_reader")
vframes, _, _ = torchvision.io.read_video('videos/SOX5yA1l24A.mp4')
print("Reference number of frames", vframes.size())

Reference number of frames torch.Size([332, 256, 340, 3])


In [34]:
from decord import VideoReader
from decord import cpu
import numpy as np
vr = VideoReader('videos/SOX5yA1l24A.mp4', ctx=cpu(0))
# a file like object works as well, for in-memory decoding
nf = len(vr)
# 1. the simplest way is to directly access frames
print("Decord VR number of frames", nf)

Decord VR number of frames 332


In [35]:
frames = []
for i in range(nf):
    # the video reader will handle seeking and skipping in the most efficient manner
    frames.append(vr[i])
print("Decord VR practical number of frames", len(frames))

Decord VR practical number of frames 332


In [36]:
frames = vr.get_batch(range(nf))
print("Decord VR-batch practical number of frames", frames.shape[0])

Decord VR-batch practical number of frames 332


## Decord VideoLoader

uris : list of str
    List of video paths.
ctx : decord.Context or list of Context
    The context to decode the video file, can be decord.cpu() or decord.gpu().
    If ctx is a list, videos will be evenly split over many ctxs.
shape : tuple
    Returned shape of the batch images, e.g., (2, 320, 240, 3) as (Batch, H, W, 3)
interval : int
    Intra-batch frame interval.
skip : int
    Inter-batch frame interval.
shuffle : int
    Shuffling strategy. Can be
    `0`:  all sequential, no seeking, following initial filename order
    `1`:  random filename order, no random access for each video, very efficient
    `2`:  random order
    `3`:  random frame access in each video only.

In [44]:
from decord import VideoLoader
from decord import cpu

In [48]:
vr = VideoLoader(['videos/SOX5yA1l24A.mp4'], shape=(50, 256, 340, 3), ctx=cpu(0), interval=0, skip=0, shuffle=1)
print(len(vr))

6


In [50]:
for i in range(len(vr)):
    frames, idx = vr.next()


AttributeError: 'NDArray' object has no attribute 'tolist'

In [53]:
a = [3, 9]
np.mean(a)

6.0

# Testing for API

In [96]:
import av
container = av.open('videos/SOX5yA1l24A.mp4')

In [74]:
print([x.type for x in list(container.streams)])

['video', 'audio']


In [138]:
container.decode(container.streams.video).__next__()
    

EOFError: [Errno 541478725] End of file

In [125]:
for frame in container.decode(container.streams.audio[0]):
    print(frame.pts)
    

In [139]:
container.seek(200000, stream=container.streams.audio[0])

In [141]:
print(container.streams.audio[0].pts)

AttributeError: pts

In [115]:
container.seek(516096, stream=container.streams.video[0])

In [93]:
keyframes = []
for packet in container.demux():
    if packet.is_keyframe:
        keyframes.append(packet.pts)
print(keyframes)

[2048, 3072, 4096, 5120, 6144, 7168, 8192, 9216, 10240, 11264, 12288, 13312, 14336, 15360, 16384, 17408, 18432, 19456, 20480, 21504, 22528, 23552, 24576, 25600, 26624, 27648, 28672, 29696, 30720, 31744, 32768, 33792, 34816, 35840, 36864, 37888, 38912, 39936, 40960, 41984, 43008, 44032, 45056, 46080, 47104, 48128, 49152, 50176, 51200, 52224, 53248, 54272, 55296, 56320, 57344, 58368, 59392, 60416, 61440, 62464, 63488, 64512, 65536, 66560, 67584, 68608, 69632, 70656, 71680, 72704, 73728, 74752, 75776, 76800, 77824, 78848, 79872, 80896, 81920, 82944, 83968, 84992, 86016, 87040, 88064, 89088, 90112, 91136, 92160, 93184, 94208, 95232, 96256, 97280, 98304, 99328, 100352, 101376, 102400, 103424, 104448, 105472, 106496, 107520, 108544, 109568, 110592, 111616, 112640, 113664, 114688, 115712, 116736, 117760, 118784, 119808, 120832, 121856, 122880, 123904, 124928, 125952, 126976, 128000, 129024, 130048, 131072, 132096, 133120, 134144, 135168, 136192, 137216, 138240, 139264, 140288, 141312, 142336,

In [134]:
import math
def _sec_to_stream(ts, stream):
    return int(math.floor(ts * (1 / stream.time_base)))

def _stream_to_sec(pts, stream):
    return float(pts * stream.time_base)

In [136]:
_sec_to_stream(20, container.streams.audio[0])

960000

In [135]:
_stream_to_sec(600000, container.streams.video[0])

20.0